In [570]:
# Imports
import os
import requests
import pandas as pd
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
from datetime import datetime
from dateutil.relativedelta import *
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
import  yfinance as yf
from transformers import pipeline
# To show a progress bar when doing lengthy operations
from tqdm.notebook import tqdm

In [525]:
# Establish constants and retrieve environment variables
ETF_FILE = './data/etf.csv'
NEWS_FILE = './data/news.csv'

# Retrieve alpaca credentials
load_dotenv()
alpaca_api_key = os.getenv('APCA-API-KEY-ID')
alpaca_secret_key = os.getenv('APCA-API-SECRET-KEY')
if not(alpaca_api_key) or not(alpaca_secret_key):
    print('Failed to load API credentials')

In [526]:
# Check if ETF data file exists, if so load the data from the file
# Otherwise retrieve the data from the yahoo finance API
etf_path = Path(ETF_FILE)
if etf_path.is_file():
    etf_df = pd.read_csv(etf_path, index_col='Date', parse_dates=True, infer_datetime_format=True)
    print(etf_df.head())

                SPY.AX     STW.AX     VAS.AX
Date                                        
2014-10-13  216.729996  48.599998  65.360001
2014-10-14  213.949997  49.119999  66.059998
2014-10-15  217.089996  49.500000  66.480003
2014-10-16  212.434738  49.549999  66.599998
2014-10-17  213.059998  49.779999  66.839996


In [527]:
# Get historical OHLCV data for target ticker if the data is not already stored in a csv
if not etf_path.is_file():
    etf_df = yf.download(
        "STW.AX, VAS.AX, SPY.AX", 
        period="max"
    )

In [528]:
# Retain the key columns for each etf
if not etf_path.is_file():
    etf_df = etf_df.drop(columns = ['Adj Close', 'High', 'Low', 'Open', 'Volume'])
    etf_df.columns = etf_df.columns.droplevel()     # Changes the multilevel indexing on columns to single level
    #  Drop all nulls
    etf_df = etf_df.dropna()
    # Save ETF data to file so that we don't have to download again when this notebook is rerun
    etf_df.to_csv(etf_path)


In [529]:
etf_df.head()

,SPY.AX,STW.AX,VAS.AX
Date,,,
2014-10-13,216.729996,48.599998,65.360001
2014-10-14,213.949997,49.119999,66.059998
2014-10-15,217.089996,49.500000,66.480003
2014-10-16,212.434738,49.549999,66.599998
2014-10-17,213.059998,49.779999,66.839996


In [530]:
# Collect the top 10 stocks of the Vangard to use as proxy for the SPY.AX ETF
# Note - hardcoding the stocks as there does not appear to be an easier way to 
# programitcally retrieve the constituent stocks
spy_top10 = [
    {
        'symbol': 'AAPL',
        'description': 'Apple'
    }, 
    {
        'symbol': 'MSFT',
        'description': 'Microsoft' 
    },
    {
        'symbol': 'AMZN',
        'description': 'Amazon' 
    },
    {
        'symbol': 'TSLA',
        'description': 'Tesla' 
    },
    {
        'symbol': 'GOOGL',
        'description': 'Google'  
    },
    {
        'symbol': 'GOOG', 
        'description': 'Google'
    },
    {
        'symbol': 'NVDA', 
        'description': 'Nvidia'
    },
    {
        'symbol': 'BRK.B', 
        'description': 'Berkshire'
    },
    {
        'symbol': 'META', 
        'description': 'Meta'
    },
    {
        'symbol': 'UNH',
        'description': 'United Health'
    }
]

In [531]:
# Collect the top 10 stocks of the Vanguard fund to use as proxy for the VAS.AX ETF
# Note - hardcoding the stocks as there does not appear to be an easier way to 
# programitcally retrieve the vanguarg constituent stocks
vas_top10 = [
    {
        'symbol': 'BHP',
        'description': 'BHP'
    }, 
    {
        'symbol': 'CBA',
        'description': 'Commonwealth Bank' 
    },
    {
        'symbol': 'CSL',
        'description': 'CSL' 
    },
    {
        'symbol': 'NAB',
        'description': 'NAB' 
    },
    {
        'symbol': 'NAB',
        'description': 'National Australia Bank'
    },
    {
        'symbol': 'WBC',
        'description': 'Westpac'  
    },
    {
        'symbol': 'ANZDA', 
        'description': 'ANZ Bank'
    },
    {
        'symbol': 'WDS', 
        'description': 'Woodside'
    },
    {
        'symbol': 'MQG', 
        'description': 'Macquarie Group'
    },
    {
        'symbol': 'WES', 
        'description': 'Wesfarmers'
    },
    {
        'symbol': 'TLS',
        'description': 'Telstra'
    }
]

In [532]:
# Collect the top 10 stocks of the ASX 200 to use as proxy for the STW.AX ETF
# Note - hardcoding the stocks as there does not appear to be an easier way to 
# programitcally retrieve the constituent stocks
stw_top10 = [
    {
        'symbol': 'BHP',
        'description': 'BHP'
    }, 
    {
        'symbol': 'CBA',
        'description': 'Commonwealth Bank' 
    },
    {
        'symbol': 'CSL',
        'description': 'CSL' 
    },
    {
        'symbol': 'NAB',
        'description': 'NAB' 
    },
    {
        'symbol': 'NAB',
        'description': 'National Australia Bank'
    },
    {
        'symbol': 'WBC',
        'description': 'Westpac'  
    },
    {
        'symbol': 'ANZDA', 
        'description': 'ANZ Bank'
    },
    {
        'symbol': 'WDS', 
        'description': 'Woodside'
    },
    {
        'symbol': 'MQG', 
        'description': 'Macquarie Group'
    },
    {
        'symbol': 'WES', 
        'description': 'Wesfarmers'
    },
    {
        'symbol': 'TLS',
        'description': 'Telstra'
    }
]

In [533]:
# Check to see if we already have some data on disk
news_path = Path(NEWS_FILE)
news_path.is_file()

True

In [534]:
def retrieve_news():

    # Create an empty dataframe to receive the news
    buzz_df = pd.DataFrame()

    # Prepare to retrieve news from alpaca
    alpaca_endpoint = 'https://data.alpaca.markets/v1beta1/news'

    alpaca_headers = {
        'Apca-Api-Key-Id': alpaca_api_key,
        'Apca-Api-Secret-Key': alpaca_secret_key
    }
    
    parameters = {
        'symbols': spy_top10[0]['symbol'],
        'start': pd.to_datetime(etf_df.index[0]).strftime('%Y-%m-%d'),
        'end': pd.to_datetime(etf_df.index[-1]).strftime('%Y-%m-%d'),
        'limit': 50,
        'include_content': False,
    }

    response = requests.get(
        url = alpaca_endpoint,
        headers = alpaca_headers,
        params = parameters
    )

    response.raise_for_status()

    # Unpack the news from response
    news_page = pd.DataFrame(response.json()['news'])

    # Create a news dataframe
    # news_df = news_page.loc[news_page['headline'].str.contains(spy_top10[0]['description'], na=False, case=False)]
    buzz_df = buzz_df.append(news_page, ignore_index=True)

    # Loop through all the remaining news data
    while(response.json()['next_page_token'] != None):
        parameters['page_token'] = response.json()['next_page_token']

        response = requests.get(
            url = alpaca_endpoint,
            headers = alpaca_headers,
            params = parameters
        )

        response.raise_for_status()

        news_page = pd.DataFrame(response.json()['news'])
        # print(f"{news_page.iloc[-1]['headline']}")
        buzz_df = buzz_df.append(news_page, ignore_index=True)
    
    # Save data to csv as downloading from alpaca can take up to 10 minutes
    buzz_df.to_csv(Path('./data/news.csv'), index=False)

    # And return the dataframe to the caller
    return buzz_df

In [535]:
# Get the news for the historical period from file or from alpaca news API
news_df = pd.DataFrame()
if news_path.is_file():
    news_df = pd.read_csv(news_path)
else:
    news_df = retrieve_news()


In [546]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21503 entries, 0 to 21502
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   author      21503 non-null  object 
 1   content     0 non-null      float64
 2   created_at  21503 non-null  object 
 3   headline    21503 non-null  object 
 4   id          21503 non-null  int64  
 5   images      21503 non-null  object 
 6   source      4250 non-null   object 
 7   summary     6226 non-null   object 
 8   symbols     21503 non-null  object 
 9   updated_at  21503 non-null  object 
 10  url         21503 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 1.8+ MB


In [559]:
# Cast `news_df` column types to proper data types so that future df merges will work
news_df = news_df.astype(
    {
        'author': 'string',
        'content': 'string',
        'created_at': 'string',
        'headline': 'string',
        'images': 'string',
        'source': 'string',
        'symbols': 'string',
        'summary': 'string',
        'updated_at': 'string',
        'url': 'string'
    }
)
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21503 entries, 0 to 21502
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   author      21503 non-null  string
 1   content     0 non-null      string
 2   created_at  21503 non-null  string
 3   headline    21503 non-null  string
 4   id          21503 non-null  int64 
 5   images      21503 non-null  string
 6   source      4250 non-null   string
 7   summary     6226 non-null   string
 8   symbols     21503 non-null  string
 9   updated_at  21503 non-null  string
 10  url         21503 non-null  string
dtypes: int64(1), string(10)
memory usage: 1.8 MB


In [562]:
len(news_df)

21503

In [ ]:
# Instantiate a sentiment analysis pipleline based on the `Transformers` model
etf_pipeline = pipeline('sentiment-analysis')

In [644]:
# Filter on news related to SPY stocks
spy_df = pd.DataFrame(columns=news_df.columns)
for i in range(len(spy_top10)):
    subset_df = news_df.loc[
        news_df['headline'].str.contains(spy_top10[i]['description'], na=False, case=False)
    ]
    spy_df = pd.merge(spy_df, subset_df, how='outer')


In [645]:
# Add new column for SPY sentiment
spy_df['SPY_Sentiment'] = ''

In [646]:
# Using the `Transformers` model, add a sentiment flag for each headline
for i, row in tqdm(spy_df.iterrows(), total=len(spy_df)):
    sentiment = etf_pipeline(row['headline'])
    if sentiment[0]['label'] == 'NEGATIVE':
        sentiment_flag = '-1'
    elif sentiment[0]['label'] == 'POSITIVE':
        sentiment_flag = '1'
    else:
        sentiment_flag = '0'
    spy_df._set_value(i, 'SPY_Sentiment', sentiment_flag) 

  0%|          | 0/13885 [00:00<?, ?it/s]

In [647]:
spy_df.head()

,author,content,created_at,headline,id,images,source,summary,symbols,updated_at,url,SPY_Sentiment
0,Chris Katje,<NA>,2023-01-03T23:54:34Z,EXCLUSIVE: Top 10 Searched Tickers On Benzinga...,30267286,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,Each trading day features hundreds of headline...,"['AAPL', 'AMAM', 'AMC', 'AMZN', 'ATNF', 'CEI',...",2023-01-03T23:54:34Z,https://www.benzinga.com/general/biotech/23/01...,-1
1,Benzinga Insights,<NA>,2023-01-03T19:00:16Z,What 15 Analyst Ratings Have To Say About Apple,30262627,[],benzinga,,['AAPL'],2023-01-03T19:00:16Z,https://www.benzinga.com/analyst-ratings/23/01...,1
2,Adam Eckert,<NA>,2023-01-03T18:26:06Z,Why Apple Stock Is Falling Today,30261999,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,Apple Inc (NASDAQ: AAPL) shares are making new...,['AAPL'],2023-01-03T18:26:06Z,https://www.benzinga.com/trading-ideas/movers/...,-1
3,Adam Eckert,<NA>,2023-01-03T16:01:56Z,Apple Tells Suppliers To Build Fewer Component...,30258995,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,Apple Inc (NASDAQ: AAPL) shares slid into the ...,"['AAPL', 'FOSL', 'HNHPF', 'QCOM', 'SSNNF', 'SW...",2023-01-03T16:01:57Z,https://www.benzinga.com/news/23/01/30258995/a...,-1
4,Benzinga Newsdesk,<NA>,2023-01-03T14:35:46Z,"Apple Said To Ask Suppliers For Fewer AirPods,...",30257892,[],benzinga,<NA>,['AAPL'],2023-01-03T14:35:46Z,https://www.benzinga.com/news/23/01/30257892/a...,-1


In [620]:
# Filter on news related to ASX-200 stocks
stw_df = pd.DataFrame(columns=news_df.columns)
for i in range(len(stw_top10)):
    subset_df = news_df.loc[

        news_df['headline'].str.contains(stw_top10[i]['description'], na=False, case=False)
    ]
    stw_df = pd.merge(stw_df, subset_df, how='outer')

stw_df.shape

(60, 11)

In [621]:
# Add new column for STW sentiment
stw_df['STW_Sentiment'] = ''

In [622]:
# Using the `Transformers` model, add a sentiment flag for each headline
for i, row in tqdm(stw_df.iterrows(), total=len(stw_df)):
    sentiment = etf_pipeline(row['headline'])
    if sentiment[0]['label'] == 'NEGATIVE':
        sentiment_flag = '-1'
    elif sentiment[0]['label'] == 'POSITIVE':
        sentiment_flag = '1'
    else:
        sentiment_flag = '0'
    stw_df._set_value(i, 'STW_Sentiment', sentiment_flag) 

  0%|          | 0/60 [00:00<?, ?it/s]

In [605]:
stw_df.head()

,author,content,created_at,headline,id,images,source,summary,symbols,updated_at,url,STW_Sentiment
0,Franca Quarneti,<NA>,2022-12-19T15:43:28Z,"This Video Game Stock Has Outperformed Meta, G...",30114070,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,"Year-to-date, Activision Blizzard (NASDAQ: ATV...","['AAPL', 'AMZN', 'ATVI', 'BRK-A', 'DIS', 'GOOG...",2022-12-19T15:50:18Z,https://www.benzinga.com/general/gaming/22/12/...,-1
1,Shanthi Rexaline,<NA>,2022-08-16T14:51:14Z,Warren Buffett's Berkshire Takes Another Bite ...,28510474,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,"Warren Buffett-owned Berkshire Hathaway, Inc....","['AAPL', 'ALLY', 'AMZN', 'ATVI', 'AXP', 'BAC',...",2022-08-16T15:07:50Z,https://www.benzinga.com/trading-ideas/long-id...,1
2,Benzinga Newsdesk,<NA>,2022-08-15T20:24:18Z,Berkshire Hathaway Reports 894.8M Shares Stake...,28503696,[],benzinga,<NA>,['AAPL'],2022-08-15T20:24:19Z,https://www.benzinga.com/news/22/08/28503696/b...,1
3,Robert Kuczmarski,<NA>,2022-08-11T17:10:25Z,These 3 Dividend Yielding Stocks Are Warren Bu...,28423455,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,When looking for consistent stocks that offer ...,"['AAPL', 'AXP', 'BAC', 'BRK-A', 'BRK-B']",2022-08-11T17:10:25Z,https://www.benzinga.com/news/large-cap/22/08/...,1
4,Shanthi Rexaline,<NA>,2022-08-06T19:55:29Z,Berkshire's Debt Investments Help Mitigate Equ...,28387417,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,Warren Buffett’s Berkshire Hathaway (NYSE: BRK...,"['AAPL', 'AXP', 'BAC', 'CVX', 'KO', 'OXY']",2022-08-06T19:55:29Z,https://www.benzinga.com/news/earnings/22/08/2...,1


In [623]:
# Filter on news related to the Vanguard ETF stocks
vas_df = pd.DataFrame(columns=news_df.columns)
for i in range(len(vas_top10)):
    subset_df = news_df.loc[
        news_df['headline'].str.contains(vas_top10[i]['description'], na=False, case=False)
    ]
    vas_df = pd.merge(vas_df, subset_df, how='outer')

vas_df.shape

(109, 11)

In [624]:
# Add new column for VAS sentiment
vas_df['VAS_Sentiment'] = ''

In [625]:
# Using the `Transformers` model, add a sentiment flag for each headline
for i, row in tqdm(vas_df.iterrows(), total=len(vas_df)):
    sentiment = etf_pipeline(row['headline'])
    if sentiment[0]['label'] == 'NEGATIVE':
        sentiment_flag = '-1'
    elif sentiment[0]['label'] == 'POSITIVE':
        sentiment_flag = '1'
    else:
        sentiment_flag = '0'
    vas_df._set_value(i, 'VAS_Sentiment', sentiment_flag) 

  0%|          | 0/109 [00:00<?, ?it/s]

In [607]:
vas_df.head()

,author,content,created_at,headline,id,images,source,summary,symbols,updated_at,url,VAS_Sentiment
0,Benzinga,<NA>,2018-02-22T16:49:39Z,"Cobalt Stocks Continue Higher Thurs., Potentia...",11243568,[],<NA>,<NA>,"['AAPL', 'BHP', 'FCX', 'VALE']",2018-02-22T16:49:39Z,https://www.benzinga.com/node/11243568,-1
1,Vandana Singh,<NA>,2022-12-28T13:02:16Z,"China Relaxes Approval Of Imported Games, Goog...",30212156,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,Reuters,"['AAPL', 'AMZN', 'BA', 'BABA', 'BNBUSD', 'BTCU...",2022-12-28T13:02:16Z,https://www.benzinga.com/news/large-cap/22/12/...,-1
2,Shivdeep Dhaliwal,<NA>,2022-12-01T01:48:47Z,"After Elon Musk's Tirade, Mark Zuckerberg Slam...",29916750,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,Meta Platforms Inc (NASDAQ: META) CEO Mark Zuc...,"['AAPL', 'GOOG', 'GOOGL', 'META']",2022-12-01T01:48:47Z,https://www.benzinga.com/news/22/11/29916750/a...,-1
3,Maureen Meehan,<NA>,2022-11-25T20:09:12Z,Louis Armstrong Loved Cannabis And Didn't Care...,29844467,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,When Jazz legend and New Orleans native Louis ...,['AAPL'],2022-11-25T20:09:12Z,https://www.benzinga.com/markets/cannabis/22/1...,-1
4,Benzinga Newsdesk,<NA>,2022-11-10T13:08:52Z,"Apple Says With Upgraded Ground Stations, Soon...",29657047,[],benzinga,<NA>,"['AAPL', 'GSAT']",2022-11-10T13:08:52Z,https://www.benzinga.com/news/22/11/29657047/a...,1


In [676]:
# Focusing on the S&P 500, use the `updated_at` column as index
spy_df.index = pd.to_datetime(spy_df['updated_at'])

In [686]:
# find the most frequent signal per day
spy_df.sort_index(inplace=True)
spy_sentiment = spy_df.groupby(spy_df.index.date)['SPY_Sentiment'].apply(lambda x: x.value_counts().sort_values(ascending=False).head(1)).to_frame()

In [687]:
spy_sentiment.reset_index(level=1, inplace=True)
spy_sentiment

,level_1,SPY_Sentiment
2015-01-02,-1,2
2015-01-03,1,2
2015-01-05,-1,2
2015-01-06,-1,3
2015-01-07,-1,2
...,...,...
2022-12-28,-1,6
2022-12-29,1,5
2022-12-30,-1,6
2023-01-01,-1,1


In [688]:
spy_sentiment = spy_sentiment.rename(columns={'SPY_Sentiment': 'Sentiment_Frequency'})
spy_sentiment = spy_sentiment.rename(columns={'level_1': 'SPY_Sentiment'})
print(spy_sentiment)

           SPY_Sentiment  Sentiment_Frequency
2015-01-02            -1                    2
2015-01-03             1                    2
2015-01-05            -1                    2
2015-01-06            -1                    3
2015-01-07            -1                    2
...                  ...                  ...
2022-12-28            -1                    6
2022-12-29             1                    5
2022-12-30            -1                    6
2023-01-01            -1                    1
2023-01-03            -1                    8

[2360 rows x 2 columns]


In [690]:
# Merge sentiment and ETF values
ETF_Sentiment = pd.concat([etf_df, spy_sentiment], axis=1)
ETF_Sentiment.tail()

,SPY.AX,STW.AX,VAS.AX,SPY_Sentiment,Sentiment_Frequency
2022-12-29,559.520020,63.320000,87.440002,1,5.0
2022-12-30,565.000000,63.509998,87.699997,-1,6.0
2023-01-01,NaN,NaN,NaN,-1,1.0
2023-01-03,561.349976,62.669998,85.820000,-1,8.0
2023-01-04,564.659973,63.500000,86.930000,NaN,NaN
